<h1>Hola muchachos y benvenido en el RECORDO LINKAGGIO<h1>
<h3>Per prima cosa dobbiamo fare il setup dell'ambiente<h3>
<h4>Nulla di complicato e se hai giá installato la libreria recordlinkage nel tuo ambiente python puoi saltare questo passaggio<h4>
<h5>Ma considera che ha anche queste dipendenze:
    numpy,
    pandas,
    scipy,
    sklearn,
    jellyfish,
    joblib,
Inoltre raccomandate sono:
    numexpr - accelerating certain numerical operations,
    bottleneck - accelerating certain types of nan evaluations<h5>


In [1]:
!pip install recordlinkage

<h2>Da qui inizia l'uso e costumo della libreria<h2>

# Record Linkage e Deduplicazione Aziende

## Introduzione  
In questo notebook, utilizziamo la libreria `recordlinkage` per identificare e unire duplicati in un dataset di aziende. Seguiremo i seguenti passaggi:  
1. **Normalizzazione dei dati**  (vedi normalize_company_name.ipynb)
2. **Blocchi per ridurre i confronti**  
3. **Confronto tra record**  
4. **Classificazione dei duplicati**  
5. **Aggregazione dei migliori dati**  

---
#### **Step 1: Importazione delle Librerie e del DataFrame**

In [ ]:
import recordlinkage
from recordlinkage import Compare
from recordlinkage.preprocessing import clean
import pandas as pd

companies_df = pd.read_csv('../aziende_normalizzate.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'normalized_company_name.csv'

#### **Step 2: Creazione del blocco per la riduzione delle coppie candidate**

Il blocco serve a ridurre il numero di confronti tra righe, raggruppando insieme quelle che potrebbero appartenere alla stessa azienda. Per esempio, bloccheremo le righe con lo stesso nome azienda, città e paese.


In [3]:
indexer = recordlinkage.Index()
indexer.block(left_on=['company_name'])  #proviamo solo su normalizzazione

candidate_pairs = indexer.index(companies_df)

print(f'Numero di coppie candidate: {len(candidate_pairs)}')

Numero di coppie candidate: 39762


#### **Sicuramente questo metodo di blocking puó esser migliorato perché penso ci perdiamo un sacco di match peró é anche vero che se hanno questi valori uguali sicuro so uguali**
#### Le possibili migliorie sono 2 o normalizziamo meglio i nomi e poi mettiamo bloking solo su nomi, oppure non usiamo blocking ma facciamo direttamente compare usandolo come blocking
---

### **Step 3: Confronto delle coppie candidate**

Una volta definite le coppie di righe candidate, possiamo procedere al confronto tra i record. Utilizzeremo la similarità di stringhe (Jaro-Winkler) per confrontare altre colonne, come `industry`, `foundation_year`, `website`, e `number_of_employees`.

In [ ]:
#TRYNG TO COMPUTE STEP 3
print(type(candidate_pairs))
print(candidate_pairs[:5])

compare = Compare()
compare.string('industry', 'industry', method='jarowinkler', label='industry')
compare.numeric('foundation_year', 'foundation_year', label='foundation_year')
compare.string('website', 'website', method='jarowinkler', label='website')
compare.numeric('number_of_employees', 'number_of_employees', label='number_of_employees')
compare.string('city', 'city', method='jarowinkler', label='city')
compare.string('country', 'country', method='jarowinkler', label='country')

sampled_candidate_pairs = candidate_pairs[:5]

candidate_pairs_index = pd.MultiIndex.from_tuples(sampled_candidate_pairs, names=['left', 'right'])

print("Indice del DataFrame:", companies_df.index[:10])
print("Indice del MultiIndex:", candidate_pairs_index[:5])

companies_df = companies_df.reset_index(drop=True)
test_pair = candidate_pairs[:1]  # Prendi solo una coppia
test_index = pd.MultiIndex.from_tuples(test_pair, names=['left', 'right'])
try:
    test_features = compare.compute(test_index, companies_df)
    print(test_features)
except Exception as e:
    print("Errore:", e)

companies_df = companies_df.fillna("")




In [4]:
def compare_pair(pair, compare, companies_df):
    return compare.compute([pair], companies_df)

compare = Compare()
compare.string('industry', 'industry', method='jarowinkler', label='industry')
compare.numeric('foundation_year', 'foundation_year', label='foundation_year')
compare.string('website', 'website', method='jarowinkler', label='website')
compare.numeric('number_of_employees', 'number_of_employees', label='number_of_employees')
compare.string('city', 'city', method='jarowinkler', label='city')
compare.string('country', 'country', method='jarowinkler', label='country')

print(f"Numero di coppie candidate: {len(candidate_pairs)}")
print("Prime 5 coppie candidate:")
print(candidate_pairs[:5])

sampled_candidate_pairs = candidate_pairs[:5]  # Limita a 100 coppie

candidate_pairs_index = pd.MultiIndex.from_tuples(sampled_candidate_pairs, names=['left', 'right'])

features = compare.compute(candidate_pairs_index, companies_df)

features = features[features.sum(axis=1) > 0.85]

print(features.head())


Numero di coppie candidate: 39762
Prime 5 coppie candidate:
MultiIndex([(1027, 1026),
            (1028, 1026),
            (1028, 1027),
            (1029, 1026),
            (1029, 1027)],
           )


: 

### **Questo non mi funiona non so se é perché il dataset é troppo grande ma muore il kernel**

---

### **Step 4: Filtraggio delle corrispondenze**

In [25]:
matches = features[features.sum(axis=1) > 2]  # La somma delle colonne è maggiore di 2

print(f'Numero di corrispondenze trovate: {len(matches)}')

matches.head()

Numero di corrispondenze trovate: 1499


,,company_name,city,country
19,18,1.0,1.0,1
35,34,1.0,1.0,1
61,60,1.0,1.0,1
141,140,1.0,1.0,1
145,144,1.0,1.0,1


### **Step 5: Cobinazione righe**
Infine, possiamo deduplicare i dati aziendali combinando le righe corrispondenti. In questo esempio, prenderemo i valori più "affidabili" (ad esempio, il valore massimo per le colonne numeriche) per ogni gruppo di corrispondenze.

In [27]:
grouped_matches = matches.groupby(level=0).agg('max')

grouped_matches.head()


,company_name,city,country
19,1.0,1.0,1
35,1.0,1.0,1
61,1.0,1.0,1
141,1.0,1.0,1
145,1.0,1.0,1


### Conclusione

Abbiamo completato il processo di **record linkage** utilizzando il blocco e il confronto delle righe aziendali. Abbiamo ridotto il numero di confronti, filtrato le corrispondenze e finalmente deduplicato i dati per ottenere un'unica riga per ogni azienda.

Puoi ora esaminare i risultati nel file `dati_deduplicati.csv` e continuare con l'analisi dei dati.

Se desideri ottimizzare ulteriormente i parametri o esplorare altre tecniche di confronto, come l'utilizzo di diverse metriche di similarità o il blocking su altre colonne, sentiti libero di adattare il codice!